In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import make_scorer
import cloudpickle
import numpy as np
from sklearn.model_selection import cross_val_score

In [ ]:
processed = Path('../data/processed')
pred = pd.read_pickle(Path(processed) / 'main_predictions.pkl')
pred = pred[(pred.b_prev_G > 50)]
# pred = pred.head(10000)
pred = pred.reset_index().set_index(['Date', 'GAME_ID', 'BAT_ID'])

In [ ]:
with open(Path('../models/logistic_model.pkl'), 'rb') as fp:
    fitted_model = cloudpickle.load(fp)

In [ ]:
x_vars = [
    'spot', 'home', 'b_pred_HPPA', 'p_pred_HPPA',
    'rating_rating_prob', 'rating_pitcher_rgs',
    'park_h_factor', 'opp_hands',
    'p_team_pred_AdjHPG', 'p_team_pred_DefEff',
    'b_team_pred_AdjHPG'
]
y_var = ['Win']
vars = x_vars + y_var

In [ ]:
def my_custom_loss_func(y_true, y_pred):
    diff = np.abs(y_true - y_pred).max()
    return np.log1p(diff)

In [ ]:
def my_custom_loss_func2(estimator, X, y):
    data = X.copy()
    outcome = y.to_frame()
    probs = estimator.predict_proba(data)
    outcome['EstProb'] = probs[:, 1]
    
    selections = outcome.sort_values(['Date', 'EstProb'], ascending=[True, False])
    selections['pick_order'] = selections.groupby(['Date']).cumcount() + 1
    selections = selections[selections.pick_order <= 2]

    return selections['Win'].mean()

In [ ]:
cross_val_score(fitted_model,
    pred[x_vars],
    pred['Win'].astype('int'),
    scoring=my_custom_loss_func2,
    cv=5,
) 

In [ ]:
pred['Win'].astype('int').mean()